In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
def str_list_to_int_list(row):
    "Конвертация str значений строк в int"
    list_int = []
    for i in row.strip('][').split():
        list_int.append(int(i.rstrip(',')))
    return list_int

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = pd.read_csv('predictions_basic.csv')
for col in result.iloc[:,1:]:
    result[col] = result.apply(lambda row: str_list_to_int_list(row[col]), axis=1)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1133249, 1944920, 12385010, 893359, 1012992]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[869741, 12352204, 9677880, 46154, 5590613]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())

    return recs.tolist()

In [7]:
%%time

items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
# берем где продажи > 0
popularity_positive = items_weights[items_weights['sales_value'] > 0]
# берем логариф продаж
items_weights.loc[items_weights['sales_value'] > 0, 'sales_sum_log'] = \
    np.log(popularity_positive['sales_value'])

# сделаем нормализацию 0-1
items_weights['sales_sum_log_norm'] = ((items_weights['sales_sum_log']-items_weights['sales_sum_log'].min())/\
                                           (items_weights['sales_sum_log'].max()-items_weights['sales_sum_log'].min()))

# сделаем переведем в вероятность 0-1
items_weights['weight'] = items_weights['sales_sum_log_norm'] / items_weights['sales_sum_log_norm'].sum()

# избавимся от NAN
items_weights.loc[np.isnan(items_weights['weight']), 'weight'] = 0

items_weights.drop('sales_value', axis=1, inplace=True)
items_weights.drop('sales_sum_log', axis=1, inplace=True)
items_weights.drop('sales_sum_log_norm', axis=1, inplace=True)


popular_recs = weighted_random_recommendation(items_weights, n=5)

popular_recs

CPU times: total: 172 ms
Wall time: 187 ms


[13007704, 13007658, 1098267, 1094488, 754343]

In [8]:
result['weighted_random_recommendation'] = result['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1133249, 1944920, 12385010, 893359, 1012992]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[852800, 1089941, 13776774, 749063, 996975]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[869741, 12352204, 9677880, 46154, 5590613]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[851648, 13039313, 1137833, 473690, 3444990]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [10]:
for col in result.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean()))

random_recommendation:  0.0010
popular_recommendation:  0.1552
itemitem:  0.1369
cosine:  0.1329
tfidf:  0.1390
own_purchases:  0.1797
weighted_random_recommendation:  0.0007


Как и следовало ожидать рекомендации на основе попоулярных товаров и собственных покупок показывают существенно лучший результат

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### 1.1 Random recommendation

In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))
result.head(2)

CPU times: total: 1.12 s
Wall time: 1.11 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940090, 824758, 7410342, 909714, 1031864]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[852800, 1089941, 13776774, 749063, 996975]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1043187, 1083446, 872177, 14050436, 952781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[851648, 13039313, 1137833, 473690, 3444990]"


### 1.2 Popularity-based recommendation

In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [18]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5000)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: total: 172 ms
Wall time: 171 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940090, 824758, 7410342, 909714, 1031864]","[6534178, 6533889, 1029743, 6534166, 1082185, ...","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[852800, 1089941, 13776774, 749063, 996975]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1043187, 1083446, 872177, 14050436, 952781]","[6534178, 6533889, 1029743, 6534166, 1082185, ...","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[851648, 13039313, 1137833, 473690, 3444990]"


In [19]:
for col in result.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean()))

random_recommendation:  0.0055
popular_recommendation:  0.1552
itemitem:  0.1369
cosine:  0.1329
tfidf:  0.1390
own_purchases:  0.1797
weighted_random_recommendation:  0.0007


In [ ]:
Как и следовало ожидать random_recommendation увеличился с 0.0010 до 0.0055, popular_recommendation остался на том же уровне - 0.1552